In [2]:
import numpy as np
import pandas as pd

In [4]:
data = np.load("mnist.npz") # loading dataset

# assign dataset
x_train = data["x_train"]
y_train = data["y_train"]

x_test = data["x_test"]
y_test = data["y_test"]

# data normalization and flattening 
x_train = x_train / 255.0
x_train = x_train.reshape(-1, 28*28)

x_test = x_test / 255.0
x_test = x_test.reshape(-1, 28*28)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [ ]:
def initialization(dimensions): # initialisation of wheights of the ANN

	#first layer
	W1 = np.random.randn(10, 784)
	b1 = np.random.randn(10, 1)

	#second layer
	W2 = np.random.randn(10, 10)
	b2 = np.random.randn(10, 1)
	return W1, b1, W2, b2

In [ ]:
def sigmoid(z):
	return 1 / (1 + np.exp(z))
def softmax(z):
	return np.exp(z) / np.sum(np.exp(z))

def forward_propagation(X, W1, b1, W2, b2):
	#layer 0
	A0 = X

	#first layer
	Z1 = W1 @ A0 + b1
	A1 = sigmoid(Z1)

	#second layer
	Z2 = W2 @ A1 + b2
	A2 = softmax(Z2)
	return A2